In [109]:
import requests
from random import uniform
import time
import numpy as np
import re
import json

from bs4 import BeautifulSoup as BS

url = 'https://hh.ru'
occupation = 'data-scientist'
suffix = '/vacancies/' + occupation
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0'}
info = []
last_page = False
page = 0

while True:
  params = {'page': str(page)}
  time.sleep(uniform(1, 3))
  response = requests.get(url + suffix, params=params, headers=headers)
  dom = BS(response.text, 'html.parser')
  vacancies = dom.find_all('div', {'class':'vacancy-serp-item-body__main-info'})
  vacancies_list = []
  for vacancy in vacancies:
    vacancy_data = {}
    vac_info = vacancy.contents[0]
    link_name = vac_info.find('div', {'class':''}).find('h3').contents[0].contents[0]
    vacancy_data['name'] = link_name.string
    vacancy_data['link'] = link_name.find('a')['href']
    vacancy_data['salary'] = {}
    try:
      salary = vac_info.find('span', {'class':'bloko-header-section-3'}).text
      s = re.findall('\d+', salary.replace('\u202f', ''))
      if 'от' in salary:
        vacancy_data['salary']['from']= int(s[0])
        vacancy_data['salary']['to'] = np.nan
        vacancy_data['salary']['currency'] = salary[salary.rfind(' '):]
      elif 'до' in salary:
        vacancy_data['salary']['from']= np.nan
        vacancy_data['salary']['to'] = int(s[0])
        vacancy_data['salary']['currency'] = salary[salary.rfind(' '):]
      else:
        vacancy_data['salary']['from'] = int(s[0])
        vacancy_data['salary']['to'] = int(s[1])
        vacancy_data['salary']['currency'] = salary[salary.rfind(' '):]
    except:
      vacancy_data['salary']['from'] = np.nan
      vacancy_data['salary']['to'] = np.nan
      vacancy_data['salary']['currency'] = np.nan

    info.append(vacancy_data)

  if dom.find('div', {'class':'pager'}).find('a', {'data-qa' : 'pager-next'}) != None:
    page += 1
  else:
    break  

with open('ds_vacancies.json', 'w') as file:
  # for vacancy in info:
  json.dump(info , file)

{'name': 'Data scientist', 'link': 'https://hh.ru/vacancy/53215762?query=data%20scientist', 'salary': {'from': nan, 'to': nan, 'currency': nan}}
{'name': 'Data scientist', 'link': 'https://hh.ru/vacancy/53092406?query=data%20scientist', 'salary': {'from': 300000, 'to': nan, 'currency': ' руб.'}}
{'name': 'Junior Data scientist/ Младший риск-аналитик', 'link': 'https://hh.ru/vacancy/53211742?query=data%20scientist', 'salary': {'from': nan, 'to': nan, 'currency': nan}}
{'name': 'Data scientist', 'link': 'https://hh.ru/vacancy/52625579?query=data%20scientist', 'salary': {'from': 150000, 'to': 220000, 'currency': ' руб.'}}
{'name': 'Data Scientist', 'link': 'https://hh.ru/vacancy/49423067?query=data%20scientist', 'salary': {'from': nan, 'to': 320000, 'currency': ' руб.'}}
{'name': 'Data scientist / Product analyst', 'link': 'https://hh.ru/vacancy/52276251?query=data%20scientist', 'salary': {'from': 200000, 'to': 250000, 'currency': ' руб.'}}
{'name': 'Data Scientist', 'link': 'https://hh.r